# Parte 3: Recolección de información
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de librerias

In [ ]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [ ]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Lectura de archivo de noticias

In [ ]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'df_consolidated_clean.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

### Paso 5: Instacia del objeto e impresión del DataFrame

In [ ]:
# Instancia del objeto
IC = IterableClass()

# 5.1. Eliminación de columna que se creo en por error "Unnamed: 0" .drop('Unnamed: 0', axis=1)
# 5.2. Reseteo de index por la acción de combinación de DataFrames .reset_index().drop(['index'], axis=1)
IC.dateframe.drop('Unnamed: 0', axis=1).reset_index().drop(['index'], axis=1)

### Paso 6: Definición de función para unificar los diferentes parrafos de la noticia

In [ ]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

### Paso 7: Definición de función para unificar los diferentes temas de la noticia

In [ ]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

### Paso 8: Recolección de información para cada noticia

In [ ]:
# 8.1 Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

In [ ]:
# PASO 8.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 8.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

In [ ]:
# PASO 8.3 Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/p')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        author_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/a/div')
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if author_a:
                    for i in author_a:
                        print(authors.append(i.text))
                else: authors.append('...')
            elif len(author_a) > 1:
                author_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[1]')
                if author_b:
                    for i in author_b:
                        print(authors.append(i.text))
                else: authors.append('...')
        else: authors.append('...')
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if date_a:
                    for i in date_a:
                        print(dates.append(i.text))
                else: dates.append('...')
            elif len(date_a) > 1:
                date_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[2]')
                if date_b:
                    for i in date_b:
                        print(dates.append(i.text))
                else: dates.append('...')
        else: dates.append('...')
        # ****************************************************************************************************************
        

        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'prose')
        if paragraphs_a: 
            if len(paragraphs_a) > 1:
                paragraphs.append(JoinParagraphs(paragraphs_a))
            else:
                paragraphs.append(paragraphs_a[0].text)
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.inline-block')
        if tags: 
            if len(tags) > 1:
                themes.append(JoinThemes(tags))
            else:
                themes.append(tags[0].text)
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

In [ ]:
df_news_clean = df.drop_duplicates(subset=["titles"])

In [ ]:
df_news_clean.to_csv("df_news_clean.csv")

In [ ]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

In [ ]:
df_news_clean